# Mercedes-Benz Greener Manufacturing
Course-end Project 1

Name-Nishant Dubey

Objective-You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Mercedes-Benz’s standards

In [1]:
# Importing the required libraries

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing the data 

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
train.columns

Index(['ID', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=378)

In [6]:
test.columns

Index(['ID', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X10',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=377)

In [7]:
print('Size of training set: {} rows and {} columns'.format(*train.shape))
print('Size of testing set: {} rows and {} columns'.format(*test.shape))

Size of training set: 4209 rows and 378 columns
Size of testing set: 4209 rows and 377 columns


In [8]:
# Collect the Y values into an array
y_train = train['y'].values

In [9]:
y_train

array([130.81,  88.53,  76.26, ..., 109.22,  87.48, 110.85])

In [10]:
# Understanding the data types:
cols = [c for c in train.columns if 'X' in c]
print('Number of faetures: {}'.format(len(cols)))
print('Feature types:')
train[cols].dtypes.value_counts()

Number of faetures: 376
Feature types:


int64     368
object      8
dtype: int64

In [11]:
# Count the data in each of the columns

counts = [[], [], []]
for c in cols:
    typ = train[c].dtype
    uniq = len(np.unique(train[c]))
    if uniq ==1:
        counts[0].append(c)
    elif uniq == 2 and typ ==np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)
print('Constant features: {} Binary feature: {} Categorical features: {}\n'
 .format(*[len(c) for c in counts]))
print('Constant features:',counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary feature: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [12]:
# Splitting the data 
 
usable_columns = list(set(train.columns) - set(['ID','y']))
y_train = train['y'].values
id_test = test['ID'].values
x_train = train[usable_columns]
x_test = test[usable_columns]

# Checking for null values and unique values for train and test data

In [13]:
def check_missing_values(df):
    if df.isnull().any().any():
        print('There are missing values in the dataframe')
    else:
        print('There are no missing values in the dataframe')

In [14]:
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


# Label Encoding the categorical values

In [15]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # column with only one 
        # value is useless so we drop it.
        x_test.drop(column, axis=1)
    if cardinality >2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

,X24,X356,X291,X274,X292,X324,X178,X325,X142,X308,...,X196,X107,X385,X187,X311,X212,X217,X164,X355,X314
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Make sure the data is changed into numerical values

print('featurectypes:')
x_train[cols].dtypes.value_counts()

featurectypes:


int64    376
dtype: int64

# Perform Dimensionality Reduction

In [17]:
n_comp = 12
pca = PCA(n_components = n_comp,random_state = 420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

# Training Using XGBoost

In [18]:
# Training Using XGBoost

import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_val,y_train,y_val = train_test_split(pca2_results_train, y_train, test_size=0.2, random_state=4242)

In [20]:
d_train = xgb.DMatrix(x_train,label = y_train)
d_val = xgb.DMatrix(x_val,label = y_val)

# dtest = xgb.DMatrix(x_test)

d_test = xgb.DMatrix(pca2_results_test)

In [21]:
params = {}
params['Objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)
watchlist = [(d_train, 'train'),(d_val,'valid')]
clf = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50,
               feval=xgb_r2_score, maximize=True, verbose_eval=10)
    

[13:09:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Objective" } are not used.

[0]	train-rmse:99.14834	train-r2:-58.35295	valid-rmse:98.26297	valid-r2:-67.63754
[10]	train-rmse:81.27653	train-r2:-38.88428	valid-rmse:80.36433	valid-r2:-44.91014
[20]	train-rmse:66.71610	train-r2:-25.87403	valid-rmse:65.77334	valid-r2:-29.75260
[30]	train-rmse:54.86913	train-r2:-17.17722	valid-rmse:53.89147	valid-r2:-19.64534
[40]	train-rmse:45.24710	train-r2:-11.36098	valid-rmse:44.22334	valid-r2:-12.90225
[50]	train-rmse:37.44856	train-r2:-7.46723	valid-rmse:36.37638	valid-r2:-8.40634
[60]	train-rmse:31.14585	train-r2:-4.85695	valid-rmse:30.02279	valid-r2:-5.40743
[70]	train-rmse:26.08417	train-r2:-3.10795	valid-rmse:24.91516	valid-r2:-3.41275
[80]	train-rmse:22.04312	train-r2:-1.93371	valid-rmse:20.83299	valid-r2:-2.08521
[90]	train-rmse:18.84671	train-r2:-1.14458	valid-rmse:17.59846	

# Predicting test_df using XGBoost

In [22]:
p_test = clf.predict(d_test)


In [23]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('test_df.csv', index = False)
sub.head()

,ID,y
0,1,83.397812
1,2,97.286064
2,3,83.171097
3,4,76.930611
4,5,112.544647
